In [1]:
import os
import random
import numpy as np
import pandas as pd
import pickle
import warnings
import math
warnings.filterwarnings("ignore")

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error

# Set deterministic behavior for CUDA (set before torch imports)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn

import optuna

# Deterministic seed
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)
    return seed

# Global device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device} with {torch.cuda.device_count()} GPU(s)")

Using device: cuda with 5 GPU(s)


### Import Data

In [2]:
# Adjust the data directory as needed
DL_DIR = "../../data/deep_learning"
# Load the regression split dictionary.
with open(f'{DL_DIR}/comb_reg_dict.pkl', 'rb') as f:
    comb_reg_dict = pickle.load(f)

with open(f'{DL_DIR}/fitbit_reg_dict.pkl', 'rb') as f:
    fitbit_reg_dict = pickle.load(f)

# Load the classification split dictionary.
with open(f'{DL_DIR}/comb_class_dict.pkl', 'rb') as f:
    comb_class_dict = pickle.load(f)

with open(f'{DL_DIR}/fitbit_class_dict.pkl', 'rb') as f:
    fitbit_class_dict = pickle.load(f) 

### Utility: compute output length after conv and pooling

In [3]:
def conv_output_length(L_in, kernel_size, stride, padding, dilation):
    return (L_in + 2 * padding - dilation * (kernel_size - 1) - 1) // stride + 1

def pool_output_length(L_in, pool_kernel):
    # Assume stride==kernel size for pooling
    return L_in // pool_kernel

# Build subject-level dataset (each row is one subject)
def create_subject_dataset(df, outcome_col="SI_mean", use_weights=True):
    """
    Aggregates records for each subject into a subject-level sample.
    If use_weights is False, then sample_weight is set to 1.0 for all subjects.
    """
    exclude_cols = ["PatientID", "timepoints", "si_kde_weight", "SI_mean", "is_SI", "SI_level"]
    predictor_cols = [col for col in df.columns if col not in (exclude_cols + [outcome_col])]
    
    subject_data = []
    for pid, group in df.groupby("PatientID"):
        group_sorted = group.sort_values("timepoints")
        X = group_sorted[predictor_cols].values.T  # shape: (n_features, 39)
        y = group_sorted[outcome_col].iloc[0]
        if use_weights and "si_kde_weight" in group_sorted.columns:
            weight = group_sorted["si_kde_weight"].iloc[0]
        else:
            weight = 1.0
        record = {"PatientID": pid, "X": X, outcome_col: y, "sample_weight": weight}
        if outcome_col == "is_SI" and "SI_mean" in group_sorted.columns:
            record["SI_mean"] = group_sorted["SI_mean"].iloc[0]
        subject_data.append(record)
    subj_df = pd.DataFrame(subject_data)
    subj_df[f"{outcome_col}_bin"] = np.round(subj_df[outcome_col]).astype(int)
    return subj_df, predictor_cols


def save_results_pickle(result_dict, model_name, use_sample_weights=True):
    """
    Saves the result pickle to a folder "search". If use_sample_weights is False,
    appends '_nw' to the filename.
    """
    save_folder = "search"
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    if use_sample_weights:
        filename = os.path.join(save_folder, f"{model_name}_deep_search_results.pkl")
    else:
        filename = os.path.join(save_folder, f"{model_name}_nw_deep_search_results.pkl")
    with open(filename, "wb") as f:
        pickle.dump(result_dict, f)
    print(f"Results saved to {filename}")
    
def get_stratified_cv_splits(df, subject_id="PatientID", target_var="SI_mean", n_splits=5):
    """
    Performs stratified K-fold cross validation at the subject level.
    
    Parameters:
      df : pandas.DataFrame
          The original dataframe containing repeated measures.
      subject_id : str
          The column name for the subject ID (e.g., "PatientID").
      target_var : str
          The target variable; for regression use "SI_mean" and for classification use "is_SI".
      n_splits : int
          Number of folds for cross validation.
    
    Returns:
      splits : list of tuples
          A list where each element is a tuple (train_df, test_df) corresponding
          to one fold. Each dataframe contains all rows (i.e. repeated measures) for the patients in that fold.
    
    Behavior:
      - Isolates unique patient IDs and their target variable by dropping duplicates.
      - If target_var is "SI_mean", creates a new column "SI_mean_levels" (rounded SI_mean).
      - Uses the resulting column as the stratification column.
      - Performs stratified K-fold CV and then subsets the original dataframe based on the patient IDs.
    """
    # Create a subject-level dataframe (unique patient IDs with their target variable)
    subject_df = df[[subject_id, target_var]].drop_duplicates(subset=[subject_id]).copy()
    
    # For regression: create a new column with the rounded SI_mean values.
    if target_var == "SI_mean":
        subject_df["SI_mean_levels"] = subject_df[target_var].round().astype(int)
        strat_col = "SI_mean_levels"
    else:
        strat_col = target_var  # For classification, use the target directly.
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    splits = []
    
    # Get the subject IDs and stratification labels
    subjects = subject_df[subject_id].values
    strat_labels = subject_df[strat_col].values
    
    # For each fold, retrieve patient IDs and then subset the original dataframe.
    for train_idx, test_idx in skf.split(subjects, strat_labels):
        train_patient_ids = subject_df.iloc[train_idx][subject_id].values
        test_patient_ids  = subject_df.iloc[test_idx][subject_id].values
        train_split = df[df[subject_id].isin(train_patient_ids)]
        test_split  = df[df[subject_id].isin(test_patient_ids)]
        splits.append((train_split, test_split))
    
    return splits




###  Deep Search Objective Functions for regression

Comb Deep Search

In [4]:
#### Updated objective_regression_comb using LSTM
def objective_regression_comb_nw(trial, data_dict):
    """
    Objective function for comb_reg using LSTM on non-weighted data.
    Searched hyperparameters: lr, dropout.
    Fixed defaults: num_epochs=10, hidden_size=64, batch_size=32.
    """
    set_seed(42)
    splits = get_stratified_cv_splits(
        data_dict['train'], subject_id="PatientID", target_var="SI_mean", n_splits=5
    )
    overall_rmse_list = []
    bin_rmse_dict = {str(b): [] for b in range(1, 6)}
    
    # Hyperparameter search
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    dropout_val = trial.suggest_float("dropout", 0.1, 0.5)
    # Fixed parameters (assumed optimal)
    num_epochs = 10
    hidden_size = 64
    batch_size = 32

    for train_split, val_split in splits:
        # Force non-weighted by setting use_weights=False
        train_df, _ = create_subject_dataset(train_split, outcome_col="SI_mean", use_weights=False)
        val_df, _   = create_subject_dataset(val_split, outcome_col="SI_mean", use_weights=False)
        
        X_train = np.stack(train_df["X"].values, axis=0)  # shape: (n_subjects, n_features, seq_len)
        y_train = train_df["SI_mean"].values
        w_train = train_df["sample_weight"].values  # all 1.0
        
        X_val = np.stack(val_df["X"].values, axis=0)
        y_val = val_df["SI_mean"].values
        
        # Transpose to (n_subjects, seq_len, n_features)
        X_train = np.transpose(X_train, (0, 2, 1))
        X_val   = np.transpose(X_val, (0, 2, 1))
        n_subjects, seq_len, input_size = X_train.shape

        # Define LSTM model for regression
        class CombLSTM(nn.Module):
            def __init__(self, input_size, hidden_size, dropout):
                super(CombLSTM, self).__init__()
                self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
                self.dropout = nn.Dropout(dropout)
                self.fc = nn.Linear(hidden_size, 1)
            def forward(self, x):
                lstm_out, (h_n, _) = self.lstm(x)
                h_last = h_n[-1]
                out = self.dropout(h_last)
                out = self.fc(out)
                return out

        model = CombLSTM(input_size, hidden_size, dropout_val).to(device)
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
        
        optimizer = optim.Adam(model.parameters(), lr=lr)
        loss_fn = nn.MSELoss(reduction="none")
        
        train_dataset = torch.utils.data.TensorDataset(
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(y_train, dtype=torch.float32).view(-1, 1),
            torch.tensor(w_train, dtype=torch.float32)
        )
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
        
        model.train()
        for epoch in range(num_epochs):
            for X_batch, y_batch, weight_batch in train_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                weight_batch = weight_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = (loss_fn(outputs, y_batch).view(-1) * weight_batch).mean()
                loss.backward()
                optimizer.step()
        
        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            preds = model(X_val_tensor).cpu().numpy()
        fold_rmse = np.sqrt(np.mean((preds - y_val.reshape(-1, 1)) ** 2))
        overall_rmse_list.append(fold_rmse)
        
        # Compute per-bin RMSE.
        for b in range(1, 6):
            idx = np.where(val_df["SI_mean_bin"].values == b)[0]
            if len(idx) > 0:
                bin_rmse = np.sqrt(np.mean((preds[idx] - y_val[idx].reshape(-1, 1)) ** 2))
                bin_rmse_dict[str(b)].append(bin_rmse)
                
    overall_mean_rmse = np.mean(overall_rmse_list)
    overall_std_rmse = np.std(overall_rmse_list)
    bin_mean_rmse = {str(b): np.mean(bin_rmse_dict[str(b)]) if len(bin_rmse_dict[str(b)]) > 0 else float('nan')
                     for b in range(1, 6)}
    bin_std_rmse = {str(b): np.std(bin_rmse_dict[str(b)]) if len(bin_rmse_dict[str(b)]) > 0 else float('nan')
                    for b in range(1, 6)}
    
    trial.set_user_attr("overall_mean_rmse", overall_mean_rmse)
    trial.set_user_attr("overall_std_rmse", overall_std_rmse)
    trial.set_user_attr("bin_mean_rmse", bin_mean_rmse)
    trial.set_user_attr("bin_std_rmse", bin_std_rmse)
    
    return overall_mean_rmse

def objective_regression_comb(trial, data_dict):
    """
    Objective function for comb_reg using LSTM.
    Tunable hyperparameters:
      - hidden_size
      - lr
      - num_epochs
      - dropout
    """
    set_seed(42)
    splits = get_stratified_cv_splits(data_dict['train'], subject_id="PatientID", target_var="SI_mean", n_splits=5)
    overall_rmse_list = []
    bin_rmse_dict = {str(b): [] for b in range(1, 6)}
    
    # Tunable hyperparameters
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    dropout_val = trial.suggest_float("dropout", 0.1, 0.5)
    num_epochs = trial.suggest_int("num_epochs", 5, 10)
    hidden_size = trial.suggest_int("hidden_size", 16, 128, step=16)
    batch_size = 32  # fixed

    for train_split, val_split in splits:
        # Build subject-level datasets for training and validation
        train_df, _ = create_subject_dataset(train_split, outcome_col="SI_mean")
        val_df, _ = create_subject_dataset(val_split, outcome_col="SI_mean")
        
        X_train = np.stack(train_df["X"].values, axis=0)  # (n_subjects, n_features, seq_len)
        y_train = train_df["SI_mean"].values
        w_train = train_df["sample_weight"].values
        
        X_val = np.stack(val_df["X"].values, axis=0)
        y_val = val_df["SI_mean"].values
        val_bins = val_df["SI_mean_bin"].values
        
        # Transpose to (n_subjects, seq_len, n_features) for LSTM
        X_train = np.transpose(X_train, (0, 2, 1))
        X_val = np.transpose(X_val, (0, 2, 1))
        n_subjects, seq_len, input_size = X_train.shape

        # Define LSTM model for regression
        class CombLSTM(nn.Module):
            def __init__(self, input_size, hidden_size, dropout):
                super(CombLSTM, self).__init__()
                self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
                self.dropout = nn.Dropout(dropout)
                self.fc = nn.Linear(hidden_size, 1)
            def forward(self, x):
                lstm_out, (h_n, _) = self.lstm(x)
                h_last = h_n[-1]  # take the last hidden state
                out = self.dropout(h_last)
                out = self.fc(out)
                return out

        model = CombLSTM(input_size, hidden_size, dropout_val).to(device)
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
        
        optimizer = optim.Adam(model.parameters(), lr=lr)
        loss_fn = nn.MSELoss(reduction="none")
        
        train_dataset = torch.utils.data.TensorDataset(
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(y_train, dtype=torch.float32).view(-1, 1),
            torch.tensor(w_train, dtype=torch.float32)
        )
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
        
        model.train()
        for epoch in range(num_epochs):
            for X_batch, y_batch, weight_batch in train_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                weight_batch = weight_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = (loss_fn(outputs, y_batch).view(-1) * weight_batch).mean()
                loss.backward()
                optimizer.step()
        
        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            preds = model(X_val_tensor).cpu().numpy()
        fold_rmse = np.sqrt(np.mean((preds - y_val.reshape(-1, 1)) ** 2))
        overall_rmse_list.append(fold_rmse)
        
        # Compute per-bin RMSE.
        for b in range(1, 6):
            idx = np.where(val_bins == b)[0]
            if len(idx) > 0:
                bin_rmse = np.sqrt(np.mean((preds[idx] - y_val[idx].reshape(-1, 1)) ** 2))
                bin_rmse_dict[str(b)].append(bin_rmse)
                
    overall_mean_rmse = np.mean(overall_rmse_list)
    overall_std_rmse = np.std(overall_rmse_list)
    bin_mean_rmse = {str(b): np.mean(bin_rmse_dict[str(b)]) if len(bin_rmse_dict[str(b)]) > 0 else float('nan')
                     for b in range(1, 6)}
    bin_std_rmse = {str(b): np.std(bin_rmse_dict[str(b)]) if len(bin_rmse_dict[str(b)]) > 0 else float('nan')
                    for b in range(1, 6)}
    
    trial.set_user_attr("overall_mean_rmse", overall_mean_rmse)
    trial.set_user_attr("overall_std_rmse", overall_std_rmse)
    trial.set_user_attr("bin_mean_rmse", bin_mean_rmse)
    trial.set_user_attr("bin_std_rmse", bin_std_rmse)
    
    return overall_mean_rmse



def run_deep_search_comb_reg(data_dict, use_sample_weights, model_name, n_trials=75):
    """
    Run deep search for comb_reg.
    Chooses the weighted or non-weighted objective based on use_sample_weights.
    """
    data_dict["use_sample_weights"] = use_sample_weights
    optuna.logging.set_verbosity(optuna.logging.WARNING)

    def print_progress(study, trial):
        print(f"{model_name} trial {len(study.trials)}/{n_trials}\r", end="", flush=True)

    study = optuna.create_study(direction="minimize")
    # Select the appropriate objective function:
    optimize_fn = objective_regression_comb if use_sample_weights else objective_regression_comb_nw

    study.optimize(lambda trial: optimize_fn(trial, data_dict),
                   n_trials=n_trials, callbacks=[print_progress])

    mean_rows = []
    std_rows = []
    for t in study.trials:
        row_mean = {
            "model": model_name,
            "type": "weighted" if use_sample_weights else "not weighted",
            "1": t.user_attrs["bin_mean_rmse"].get("1", float('nan')),
            "2": t.user_attrs["bin_mean_rmse"].get("2", float('nan')),
            "3": t.user_attrs["bin_mean_rmse"].get("3", float('nan')),
            "4": t.user_attrs["bin_mean_rmse"].get("4", float('nan')),
            "5": t.user_attrs["bin_mean_rmse"].get("5", float('nan')),
            "overall": t.user_attrs["overall_mean_rmse"],
            "config": t.params
        }
        row_std = {
            "model": model_name,
            "type": "weighted" if use_sample_weights else "not weighted",
            "1": t.user_attrs["bin_std_rmse"].get("1", float('nan')),
            "2": t.user_attrs["bin_std_rmse"].get("2", float('nan')),
            "3": t.user_attrs["bin_std_rmse"].get("3", float('nan')),
            "4": t.user_attrs["bin_std_rmse"].get("4", float('nan')),
            "5": t.user_attrs["bin_std_rmse"].get("5", float('nan')),
            "overall": t.user_attrs["overall_std_rmse"],
            "config": t.params
        }
        mean_rows.append(row_mean)
        std_rows.append(row_std)

    columns = ["model", "type", "1", "2", "3", "4", "5", "overall", "config"]
    mean_df = pd.DataFrame(mean_rows, columns=columns)
    std_df = pd.DataFrame(std_rows, columns=columns)

    best_trial = study.best_trial
    optimal_configuration = {
        "value": best_trial.value,
        "params": best_trial.params,
        "user_attrs": best_trial.user_attrs
    }

    result_dict = {
        "mean_rmse": mean_df,
        "std_rmse": std_df,
        "optimal_configuration": optimal_configuration
    }
    save_results_pickle(result_dict, model_name, use_sample_weights)
    return result_dict


Fitbit Deep Search

In [5]:
#### Updated objective_regression_fitbit using LSTM
def objective_regression_fitbit_nw(trial, data_dict):
    """
    Objective function for fitbit_reg using LSTM on non-weighted data.
    Searched hyperparameters: lr, n_fc_layers.
    Fixed parameters: dropout=0.3, use_regularization=False, bidirectional=False,
                      num_epochs=10, batch_size=32, hidden_size=32.
    """
    set_seed(42)
    splits = get_stratified_cv_splits(
        data_dict['train'], subject_id="PatientID", target_var="SI_mean", n_splits=5
    )
    overall_rmse_list = []
    bin_rmse_dict = {str(b): [] for b in range(1, 6)}

    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    n_fc_layers = trial.suggest_int("n_fc_layers", 1, 3)
    
    # Fixed parameters
    dropout_val = 0.3
    use_regularization = False
    bidirectional = False
    num_epochs = 10
    batch_size = 32
    hidden_size = 32

    for train_split, val_split in splits:
        # Force non-weighted
        train_df, _ = create_subject_dataset(train_split, outcome_col="SI_mean", use_weights=False)
        val_df, _   = create_subject_dataset(val_split, outcome_col="SI_mean", use_weights=False)
        
        X_train = np.stack(train_df["X"].values, axis=0)
        y_train = train_df["SI_mean"].values
        w_train = train_df["sample_weight"].values  # should be all 1.0
        
        X_val = np.stack(val_df["X"].values, axis=0)
        y_val = val_df["SI_mean"].values
        
        X_train = np.transpose(X_train, (0, 2, 1))
        X_val   = np.transpose(X_val, (0, 2, 1))
        n_subjects, seq_len, input_size = X_train.shape

        class FitbitRegLSTM(nn.Module):
            def __init__(self, input_size, hidden_size, dropout, bidirectional, n_fc_layers):
                super(FitbitRegLSTM, self).__init__()
                self.bidirectional = bidirectional
                self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                                    batch_first=True, bidirectional=bidirectional)
                fc_in_dim = hidden_size * (2 if bidirectional else 1)
                fc_layers = []
                for i in range(n_fc_layers - 1):
                    fc_layers.append(nn.Linear(fc_in_dim, fc_in_dim))
                    fc_layers.append(nn.ReLU())
                fc_layers.append(nn.Linear(fc_in_dim, 1))
                self.fc = nn.Sequential(*fc_layers)
                self.dropout = nn.Dropout(dropout)
            def forward(self, x):
                lstm_out, (h_n, _) = self.lstm(x)
                if self.bidirectional:
                    h_last = torch.cat((h_n[-2], h_n[-1]), dim=1)
                else:
                    h_last = h_n[-1]
                out = self.dropout(h_last)
                out = self.fc(out)
                return out

        model = FitbitRegLSTM(input_size, hidden_size, dropout_val, bidirectional, n_fc_layers).to(device)
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
        
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4 if use_regularization else 0.0)
        loss_fn = nn.MSELoss(reduction="none")
        
        train_dataset = torch.utils.data.TensorDataset(
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(y_train, dtype=torch.float32).view(-1, 1),
            torch.tensor(w_train, dtype=torch.float32)
        )
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        model.train()
        for epoch in range(num_epochs):
            for X_batch, y_batch, weight_batch in train_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                weight_batch = weight_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = (loss_fn(outputs, y_batch).view(-1) * weight_batch).mean()
                loss.backward()
                optimizer.step()
        
        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            preds = model(X_val_tensor).cpu().numpy()
        fold_rmse = np.sqrt(np.mean((preds - y_val.reshape(-1, 1)) ** 2))
        overall_rmse_list.append(fold_rmse)
        
        for b in range(1, 6):
            idx = np.where(val_df["SI_mean_bin"].values == b)[0]
            if len(idx) > 0:
                bin_rmse = np.sqrt(np.mean((preds[idx] - y_val[idx].reshape(-1, 1)) ** 2))
                bin_rmse_dict[str(b)].append(bin_rmse)
    
    overall_mean_rmse = np.mean(overall_rmse_list)
    overall_std_rmse = np.std(overall_rmse_list)
    bin_mean_rmse = {str(b): np.mean(bin_rmse_dict[str(b)]) if len(bin_rmse_dict[str(b)]) > 0 else float('nan')
                     for b in range(1, 6)}
    bin_std_rmse = {str(b): np.std(bin_rmse_dict[str(b)]) if len(bin_rmse_dict[str(b)]) > 0 else float('nan')
                    for b in range(1, 6)}
    
    trial.set_user_attr("overall_mean_rmse", overall_mean_rmse)
    trial.set_user_attr("overall_std_rmse", overall_std_rmse)
    trial.set_user_attr("bin_mean_rmse", bin_mean_rmse)
    trial.set_user_attr("bin_std_rmse", bin_std_rmse)
    
    return overall_mean_rmse

def objective_regression_fitbit(trial, data_dict):
    """
    Objective function for fitbit_reg using LSTM.
    Tunable hyperparameters:
      - dropout
      - lr
      - use_regularization
      - bidirectional
      - num_epochs
      - n_fc_layers
      - batch_size
    """
    set_seed(42)
    splits = get_stratified_cv_splits(data_dict['train'], subject_id="PatientID", target_var="SI_mean", n_splits=5)
    overall_rmse_list = []
    bin_rmse_dict = {str(b): [] for b in range(1, 6)}

    # Tunable hyperparameters
    dropout_val = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    use_regularization = trial.suggest_categorical("use_regularization", [True, False])
    bidirectional = trial.suggest_categorical("bidirectional", [True, False])
    num_epochs = trial.suggest_int("num_epochs", 5, 10)
    n_fc_layers = trial.suggest_int("n_fc_layers", 1, 3)
    batch_size = trial.suggest_int("batch_size", 16, 64, step=16)
    
    # Set a default hidden_size (could be set to a fixed value)
    hidden_size = 32

    for train_split, val_split in splits:
        train_df, _ = create_subject_dataset(train_split, outcome_col="SI_mean")
        val_df, _ = create_subject_dataset(val_split, outcome_col="SI_mean")
        
        X_train = np.stack(train_df["X"].values, axis=0)  # (n_subjects, n_features, seq_len)
        y_train = train_df["SI_mean"].values
        w_train = train_df["sample_weight"].values
        
        X_val = np.stack(val_df["X"].values, axis=0)
        y_val = val_df["SI_mean"].values
        val_bins = val_df["SI_mean_bin"].values
        
        # Transpose to (n_subjects, seq_len, n_features)
        X_train = np.transpose(X_train, (0, 2, 1))
        X_val = np.transpose(X_val, (0, 2, 1))
        n_subjects, seq_len, input_size = X_train.shape

        # Define LSTM model for fitbit regression
        class FitbitRegLSTM(nn.Module):
            def __init__(self, input_size, hidden_size, dropout, bidirectional, n_fc_layers):
                super(FitbitRegLSTM, self).__init__()
                self.bidirectional = bidirectional
                self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                                    batch_first=True, bidirectional=bidirectional)
                fc_in_dim = hidden_size * (2 if bidirectional else 1)
                fc_layers = []
                # Build a stack of fully-connected layers
                for i in range(n_fc_layers - 1):
                    fc_layers.append(nn.Linear(fc_in_dim, fc_in_dim))
                    fc_layers.append(nn.ReLU())
                fc_layers.append(nn.Linear(fc_in_dim, 1))
                self.fc = nn.Sequential(*fc_layers)
                self.dropout = nn.Dropout(dropout)
            def forward(self, x):
                lstm_out, (h_n, _) = self.lstm(x)
                if self.bidirectional:
                    h_last = torch.cat((h_n[-2], h_n[-1]), dim=1)
                else:
                    h_last = h_n[-1]
                out = self.dropout(h_last)
                out = self.fc(out)
                return out

        model = FitbitRegLSTM(input_size, hidden_size, dropout_val, bidirectional, n_fc_layers).to(device)
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
        
        weight_decay = 1e-4 if use_regularization else 0.0
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        loss_fn = nn.MSELoss(reduction="none")
        
        train_dataset = torch.utils.data.TensorDataset(
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(y_train, dtype=torch.float32).view(-1, 1),
            torch.tensor(w_train, dtype=torch.float32)
        )
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        model.train()
        for epoch in range(num_epochs):
            for X_batch, y_batch, weight_batch in train_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                weight_batch = weight_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = (loss_fn(outputs, y_batch).view(-1) * weight_batch).mean()
                loss.backward()
                optimizer.step()
        
        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            preds = model(X_val_tensor).cpu().numpy()
        fold_rmse = np.sqrt(np.mean((preds - y_val.reshape(-1, 1)) ** 2))
        overall_rmse_list.append(fold_rmse)
        
        for b in range(1, 6):
            idx = np.where(val_bins == b)[0]
            if len(idx) > 0:
                bin_rmse = np.sqrt(np.mean((preds[idx] - y_val[idx].reshape(-1, 1)) ** 2))
                bin_rmse_dict[str(b)].append(bin_rmse)
    
    overall_mean_rmse = np.mean(overall_rmse_list)
    overall_std_rmse = np.std(overall_rmse_list)
    bin_mean_rmse = {str(b): np.mean(bin_rmse_dict[str(b)]) if len(bin_rmse_dict[str(b)]) > 0 else float('nan')
                     for b in range(1, 6)}
    bin_std_rmse = {str(b): np.std(bin_rmse_dict[str(b)]) if len(bin_rmse_dict[str(b)]) > 0 else float('nan')
                    for b in range(1, 6)}
    
    trial.set_user_attr("overall_mean_rmse", overall_mean_rmse)
    trial.set_user_attr("overall_std_rmse", overall_std_rmse)
    trial.set_user_attr("bin_mean_rmse", bin_mean_rmse)
    trial.set_user_attr("bin_std_rmse", bin_std_rmse)
    
    return overall_mean_rmse

def run_deep_search_fitbit_reg(data_dict, use_sample_weights, model_name, n_trials=75):
    """
    Run deep search for fitbit_reg.
    Chooses weighted vs. non-weighted objective based on use_sample_weights.
    """
    data_dict["use_sample_weights"] = use_sample_weights
    optuna.logging.set_verbosity(optuna.logging.WARNING)

    def print_progress(study, trial):
        print(f"{model_name} trial {len(study.trials)}/{n_trials}\r", end="", flush=True)

    study = optuna.create_study(direction="minimize")
    optimize_fn = objective_regression_fitbit if use_sample_weights else objective_regression_fitbit_nw

    study.optimize(lambda trial: optimize_fn(trial, data_dict),
                   n_trials=n_trials, callbacks=[print_progress])

    mean_rows = []
    std_rows = []
    for t in study.trials:
        row_mean = {
            "model": model_name,
            "type": "weighted" if use_sample_weights else "not weighted",
            "1": t.user_attrs["bin_mean_rmse"].get("1", float('nan')),
            "2": t.user_attrs["bin_mean_rmse"].get("2", float('nan')),
            "3": t.user_attrs["bin_mean_rmse"].get("3", float('nan')),
            "4": t.user_attrs["bin_mean_rmse"].get("4", float('nan')),
            "5": t.user_attrs["bin_mean_rmse"].get("5", float('nan')),
            "overall": t.user_attrs["overall_mean_rmse"],
            "config": t.params
        }
        row_std = {
            "model": model_name,
            "type": "weighted" if use_sample_weights else "not weighted",
            "1": t.user_attrs["bin_std_rmse"].get("1", float('nan')),
            "2": t.user_attrs["bin_std_rmse"].get("2", float('nan')),
            "3": t.user_attrs["bin_std_rmse"].get("3", float('nan')),
            "4": t.user_attrs["bin_std_rmse"].get("4", float('nan')),
            "5": t.user_attrs["bin_std_rmse"].get("5", float('nan')),
            "overall": t.user_attrs["overall_std_rmse"],
            "config": t.params
        }
        mean_rows.append(row_mean)
        std_rows.append(row_std)

    columns = ["model", "type", "1", "2", "3", "4", "5", "overall", "config"]
    mean_df = pd.DataFrame(mean_rows, columns=columns)
    std_df = pd.DataFrame(std_rows, columns=columns)

    best_trial = study.best_trial
    optimal_configuration = {
        "value": best_trial.value,
        "params": best_trial.params,
        "user_attrs": best_trial.user_attrs
    }

    result_dict = {
        "mean_rmse": mean_df,
        "std_rmse": std_df,
        "optimal_configuration": optimal_configuration
    }
    save_results_pickle(result_dict, model_name, use_sample_weights)
    return result_dict

### Run deep search for regression with weights

In [15]:
# Run deep search for COMB dataset (n_trials determines how many steps/trials are executed)
results_comb_reg = run_deep_search_comb_reg(comb_reg_dict, use_sample_weights=True, model_name="comb_reg_deep", n_trials=75)
results_comb_reg["mean_rmse"]


Results saved to search/comb_reg_deep_deep_search_results.pkl


,model,type,1,2,3,4,5,overall,config
0,comb_reg_deep,weighted,0.974058,1.867155,2.886802,3.848536,4.840912,1.344001,"{'lr': 1.086808570211826e-05, 'dropout': 0.307..."
1,comb_reg_deep,weighted,1.587175,0.996586,0.706037,1.324154,2.083702,1.501365,"{'lr': 0.008965726470407422, 'dropout': 0.4961..."
2,comb_reg_deep,weighted,0.755983,0.913275,1.819670,2.243301,3.014674,0.893202,"{'lr': 6.950816077040568e-05, 'dropout': 0.117..."
3,comb_reg_deep,weighted,1.838039,1.134017,0.573269,0.959681,1.679585,1.722771,"{'lr': 0.0007935277671176808, 'dropout': 0.335..."
4,comb_reg_deep,weighted,1.837981,1.205668,0.708730,1.109147,1.843238,1.737177,"{'lr': 0.00596797524338459, 'dropout': 0.13007..."
...,...,...,...,...,...,...,...,...,...
70,comb_reg_deep,weighted,1.672951,1.034426,0.691218,1.195380,1.943159,1.575959,"{'lr': 0.0015556261807786443, 'dropout': 0.267..."
71,comb_reg_deep,weighted,0.521678,1.114152,2.182451,2.789847,3.636688,0.839580,"{'lr': 6.291552319554909e-05, 'dropout': 0.328..."
72,comb_reg_deep,weighted,0.519888,1.220057,2.304465,2.952493,3.807083,0.877323,"{'lr': 5.87281898959135e-05, 'dropout': 0.3183..."
73,comb_reg_deep,weighted,0.981837,0.797988,1.522853,1.876069,2.645501,1.016205,"{'lr': 8.117895348975394e-05, 'dropout': 0.297..."


In [13]:
# Run deep search for FITBIT dataset
results_fitbit_reg = run_deep_search_fitbit_reg(fitbit_reg_dict, use_sample_weights=True, model_name="fitbit_reg_deep", n_trials=75)
results_fitbit_reg["mean_rmse"]



Results saved to search/fitbit_reg_deep_deep_search_results.pkl


,model,type,1,2,3,4,5,overall,config
0,comb_reg_deep,weighted,1.886885,1.180709,0.673292,1.038544,1.826427,1.772961,"{'lr': 0.009716870066016842, 'dropout': 0.4663..."
1,comb_reg_deep,weighted,0.950231,1.832606,2.877450,3.807033,4.802780,1.320486,"{'lr': 1.1799543502498368e-05, 'dropout': 0.16..."
2,comb_reg_deep,weighted,0.799951,1.677605,2.741591,3.632390,4.671007,1.186541,"{'lr': 4.674540872903447e-05, 'dropout': 0.141..."
3,comb_reg_deep,weighted,0.933308,1.817364,2.871818,3.748563,4.689571,1.303411,"{'lr': 1.9436751746165336e-05, 'dropout': 0.31..."
4,comb_reg_deep,weighted,1.806186,1.120519,0.648974,0.986284,1.797882,1.696076,"{'lr': 0.0008730673512666253, 'dropout': 0.121..."
...,...,...,...,...,...,...,...,...,...
70,comb_reg_deep,weighted,0.446178,1.230032,2.281848,3.094991,3.807506,0.851752,"{'lr': 9.598696524308393e-05, 'dropout': 0.224..."
71,comb_reg_deep,weighted,0.457043,0.921864,1.947518,2.670598,3.596875,0.749002,"{'lr': 0.00012727545530741416, 'dropout': 0.25..."
72,comb_reg_deep,weighted,0.557769,0.790508,1.737624,2.486406,3.167749,0.757053,"{'lr': 0.0001334513928577592, 'dropout': 0.275..."
73,comb_reg_deep,weighted,0.594035,0.755733,1.711971,2.395836,3.340215,0.769571,"{'lr': 0.00014515083939798962, 'dropout': 0.29..."


### Run deep search for regression without weights

In [16]:
# Run deep search for COMB dataset (n_trials determines how many steps/trials are executed)
results_comb_reg = run_deep_search_comb_reg(comb_reg_dict, use_sample_weights=False, model_name="comb_reg_deep", n_trials=75)
results_comb_reg["mean_rmse"]


Results saved to search/comb_reg_deep_nw_deep_search_results.pkl


,model,type,1,2,3,4,5,overall,config
0,comb_reg_deep,not weighted,0.762167,1.664520,2.705903,3.673872,4.643009,1.163452,"{'lr': 1.0784117086383168e-05, 'dropout': 0.17..."
1,comb_reg_deep,not weighted,0.255529,0.711054,1.682638,2.636605,3.700819,0.602227,"{'lr': 0.0003385621410648702, 'dropout': 0.414..."
2,comb_reg_deep,not weighted,0.394708,0.693075,1.647597,2.524740,3.587499,0.648502,"{'lr': 0.0043696633242284386, 'dropout': 0.208..."
3,comb_reg_deep,not weighted,0.324434,0.684831,1.657795,2.564347,3.632518,0.617927,"{'lr': 0.0007450940369219274, 'dropout': 0.224..."
4,comb_reg_deep,not weighted,0.767520,1.669812,2.711057,3.679885,4.650118,1.168072,"{'lr': 1.046526095246742e-05, 'dropout': 0.157..."
...,...,...,...,...,...,...,...,...,...
70,comb_reg_deep,not weighted,0.274251,0.731098,1.722326,2.650167,3.622489,0.613389,"{'lr': 7.447390323461143e-05, 'dropout': 0.229..."
71,comb_reg_deep,not weighted,0.255647,0.702178,1.680308,2.614226,3.665305,0.597130,"{'lr': 0.00018275118830783695, 'dropout': 0.21..."
72,comb_reg_deep,not weighted,0.255699,0.702785,1.679002,2.614845,3.671535,0.597397,"{'lr': 0.00020276656894787232, 'dropout': 0.21..."
73,comb_reg_deep,not weighted,0.255981,0.704251,1.678937,2.616246,3.678460,0.598121,"{'lr': 0.0002289303718206388, 'dropout': 0.209..."


In [17]:
# Run deep search for FITBIT dataset
results_fitbit_reg = run_deep_search_fitbit_reg(fitbit_reg_dict, use_sample_weights=False, model_name="fitbit_reg_deep", n_trials=75)
results_fitbit_reg["mean_rmse"]



Results saved to search/fitbit_reg_deep_nw_deep_search_results.pkl


,model,type,1,2,3,4,5,overall,config
0,comb_reg_deep,not weighted,0.762167,1.664520,2.705903,3.673872,4.643009,1.163452,"{'lr': 1.0784117086383168e-05, 'dropout': 0.17..."
1,comb_reg_deep,not weighted,0.255529,0.711054,1.682638,2.636605,3.700819,0.602227,"{'lr': 0.0003385621410648702, 'dropout': 0.414..."
2,comb_reg_deep,not weighted,0.394708,0.693075,1.647597,2.524740,3.587499,0.648502,"{'lr': 0.0043696633242284386, 'dropout': 0.208..."
3,comb_reg_deep,not weighted,0.324434,0.684831,1.657795,2.564347,3.632518,0.617927,"{'lr': 0.0007450940369219274, 'dropout': 0.224..."
4,comb_reg_deep,not weighted,0.767520,1.669812,2.711057,3.679885,4.650118,1.168072,"{'lr': 1.046526095246742e-05, 'dropout': 0.157..."
...,...,...,...,...,...,...,...,...,...
70,comb_reg_deep,not weighted,0.274251,0.731098,1.722326,2.650167,3.622489,0.613389,"{'lr': 7.447390323461143e-05, 'dropout': 0.229..."
71,comb_reg_deep,not weighted,0.255647,0.702178,1.680308,2.614226,3.665305,0.597130,"{'lr': 0.00018275118830783695, 'dropout': 0.21..."
72,comb_reg_deep,not weighted,0.255699,0.702785,1.679002,2.614845,3.671535,0.597397,"{'lr': 0.00020276656894787232, 'dropout': 0.21..."
73,comb_reg_deep,not weighted,0.255981,0.704251,1.678937,2.616246,3.678460,0.598121,"{'lr': 0.0002289303718206388, 'dropout': 0.209..."


###  Deep Search Objective Functions for classifcation

### Classification Deep Search

Comb

In [10]:
#### Updated objective_classification_comb using LSTM
def objective_classification_comb_nw(trial, data_dict):
    """
    Objective function for comb_class using LSTM on non-weighted data.
    Searched hyperparameters: dropout, lr, hidden_size, n_layers, bidirection.
    Fixed parameters: batch_size=32, epochs=10.
    """
    set_seed(42)
    splits = get_stratified_cv_splits(
        data_dict['train'], subject_id="PatientID", target_var="is_SI", n_splits=5
    )
    
    overall_acc_list = []
    overall_sens_list = []
    overall_spec_list = []
    
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    dropout_val = trial.suggest_float("dropout", 0.1, 0.5)
    hidden_size = trial.suggest_int("hidden_size", 16, 128, step=16)
    n_layers = trial.suggest_int("n_layers", 1, 3)
    bidirectional = trial.suggest_categorical("bidirection", [True, False])
    batch_size = 32

    for train_split, val_split in splits:
        # Force non-weighted for classification as well
        train_df, _ = create_subject_dataset(train_split, outcome_col="is_SI", use_weights=False)
        val_df, _   = create_subject_dataset(val_split, outcome_col="is_SI", use_weights=False)
        
        X_train = np.stack(train_df["X"].values, axis=0)
        y_train = train_df["is_SI"].values.astype(np.float32)
        
        X_val = np.stack(val_df["X"].values, axis=0)
        y_val = val_df["is_SI"].values.astype(np.float32)
        
        X_train = np.transpose(X_train, (0, 2, 1))
        X_val   = np.transpose(X_val, (0, 2, 1))
        n_subjects, seq_len, input_size = X_train.shape

        class CombClassLSTM(nn.Module):
            def __init__(self, input_size, hidden_size, dropout, n_layers, bidirectional):
                super(CombClassLSTM, self).__init__()
                self.bidirectional = bidirectional
                self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=n_layers,
                                    batch_first=True, bidirectional=bidirectional)
                fc_in_dim = hidden_size * (2 if bidirectional else 1)
                self.dropout = nn.Dropout(dropout)
                self.fc = nn.Linear(fc_in_dim, 1)
            def forward(self, x):
                lstm_out, (h_n, _) = self.lstm(x)
                if self.bidirectional:
                    h_last = torch.cat((h_n[-2], h_n[-1]), dim=1)
                else:
                    h_last = h_n[-1]
                out = self.dropout(h_last)
                out = self.fc(out)
                return out
        
        model = CombClassLSTM(input_size, hidden_size, dropout_val, n_layers, bidirectional).to(device)
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
        
        optimizer = optim.Adam(model.parameters(), lr=lr)
        loss_fn = nn.BCEWithLogitsLoss()
        
        train_dataset = torch.utils.data.TensorDataset(
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
        )
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
        
        model.train()
        for epoch in range(10):  # fixed epochs
            for X_batch, y_batch in train_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = loss_fn(outputs, y_batch)
                loss.backward()
                optimizer.step()
        
        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            logits = model(X_val_tensor)
            probs = torch.sigmoid(logits).cpu().numpy().reshape(-1)
            preds = (probs >= 0.5).astype(np.float32)
        fold_acc = np.mean(preds == y_val)
        TP = np.sum((preds == 1) & (y_val == 1))
        FN = np.sum((preds == 0) & (y_val == 1))
        sensitivity = TP / (TP + FN) if (TP + FN) > 0 else np.nan
        TN = np.sum((preds == 0) & (y_val == 0))
        FP = np.sum((preds == 1) & (y_val == 0))
        specificity = TN / (TN + FP) if (TN + FP) > 0 else np.nan
        
        overall_acc_list.append(fold_acc)
        overall_sens_list.append(sensitivity)
        overall_spec_list.append(specificity)
    
    mean_acc = np.mean(overall_acc_list)
    std_acc = np.std(overall_acc_list)
    mean_sens = np.mean(overall_sens_list)
    std_sens = np.std(overall_sens_list)
    mean_spec = np.mean(overall_spec_list)
    std_spec = np.std(overall_spec_list)
    
    trial.set_user_attr("overall_acc_mean", mean_acc)
    trial.set_user_attr("overall_acc_std", std_acc)
    trial.set_user_attr("sensitivity_mean", mean_sens)
    trial.set_user_attr("sensitivity_std", std_sens)
    trial.set_user_attr("specificity_mean", mean_spec)
    trial.set_user_attr("specificity_std", std_spec)
    
    return 1 - mean_acc

def objective_classification_comb(trial, data_dict):
    """
    Objective function for comb_class using LSTM.
    Tunable hyperparameters:
      - dropout
      - lr
      - num_epochs
      - hidden_size
      - bidirectional
    """
    set_seed(42)
    splits = get_stratified_cv_splits(data_dict['train'], subject_id="PatientID", target_var="is_SI", n_splits=5)
    
    overall_acc_list = []
    overall_sens_list = []
    overall_spec_list = []
    
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    dropout_val = trial.suggest_float("dropout", 0.1, 0.5)
    num_epochs = trial.suggest_int("num_epochs", 5, 10)
    hidden_size = trial.suggest_int("hidden_size", 16, 128, step=16)
    bidirectional = trial.suggest_categorical("bidirectional", [True, False])
    batch_size = trial.suggest_int("batch_size", 16, 64, step=16)
    
    for train_split, val_split in splits:
        train_df, _ = create_subject_dataset(train_split, outcome_col="is_SI")
        val_df, _ = create_subject_dataset(val_split, outcome_col="is_SI")
        
        X_train = np.stack(train_df["X"].values, axis=0)  # (n_subjects, n_features, seq_len)
        y_train = train_df["is_SI"].values.astype(np.float32)
        
        X_val = np.stack(val_df["X"].values, axis=0)
        y_val = val_df["is_SI"].values.astype(np.float32)
        
        # Transpose for LSTM: (n_subjects, seq_len, n_features)
        X_train = np.transpose(X_train, (0, 2, 1))
        X_val = np.transpose(X_val, (0, 2, 1))
        n_subjects, seq_len, input_size = X_train.shape

        # Define LSTM model for classification
        class CombClassLSTM(nn.Module):
            def __init__(self, input_size, hidden_size, dropout, bidirectional):
                super(CombClassLSTM, self).__init__()
                self.bidirectional = bidirectional
                self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True, bidirectional=bidirectional)
                fc_in_dim = hidden_size * (2 if bidirectional else 1)
                self.dropout = nn.Dropout(dropout)
                self.fc = nn.Linear(fc_in_dim, 1)
            def forward(self, x):
                lstm_out, (h_n, _) = self.lstm(x)
                if self.bidirectional:
                    h_last = torch.cat((h_n[-2], h_n[-1]), dim=1)
                else:
                    h_last = h_n[-1]
                out = self.dropout(h_last)
                out = self.fc(out)
                return out
        
        model = CombClassLSTM(input_size, hidden_size, dropout_val, bidirectional).to(device)
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
        
        optimizer = optim.Adam(model.parameters(), lr=lr)
        loss_fn = nn.BCEWithLogitsLoss()
        
        train_dataset = torch.utils.data.TensorDataset(
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
        )
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
        
        model.train()
        for epoch in range(num_epochs):
            for X_batch, y_batch in train_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = loss_fn(outputs, y_batch)
                loss.backward()
                optimizer.step()
        
        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            logits = model(X_val_tensor)
            probs = torch.sigmoid(logits).cpu().numpy().reshape(-1)
            preds = (probs >= 0.5).astype(np.float32)
        fold_acc = np.mean(preds == y_val)
        TP = np.sum((preds == 1) & (y_val == 1))
        FN = np.sum((preds == 0) & (y_val == 1))
        sensitivity = TP / (TP + FN) if (TP + FN) > 0 else np.nan
        TN = np.sum((preds == 0) & (y_val == 0))
        FP = np.sum((preds == 1) & (y_val == 0))
        specificity = TN / (TN + FP) if (TN + FP) > 0 else np.nan
        
        overall_acc_list.append(fold_acc)
        overall_sens_list.append(sensitivity)
        overall_spec_list.append(specificity)
    
    mean_acc = np.mean(overall_acc_list)
    std_acc = np.std(overall_acc_list)
    mean_sens = np.mean(overall_sens_list)
    std_sens = np.std(overall_sens_list)
    mean_spec = np.mean(overall_spec_list)
    std_spec = np.std(overall_spec_list)
    
    trial.set_user_attr("overall_acc_mean", mean_acc)
    trial.set_user_attr("overall_acc_std", std_acc)
    trial.set_user_attr("sensitivity_mean", mean_sens)
    trial.set_user_attr("sensitivity_std", std_sens)
    trial.set_user_attr("specificity_mean", mean_spec)
    trial.set_user_attr("specificity_std", std_spec)
    
    return 1 - mean_acc  # minimization objective


def run_deep_search_comb_class(data_dict, use_sample_weights, model_name, n_trials=75):
    """
    Run deep search for comb_class.
    Selects the proper objective function based on use_sample_weights.
    """
    data_dict["use_sample_weights"] = use_sample_weights
    optuna.logging.set_verbosity(optuna.logging.WARNING)

    def print_progress(study, trial):
        print(f"{model_name} trial {len(study.trials)}/{n_trials}\r", end="", flush=True)

    study = optuna.create_study(direction="minimize")
    optimize_fn = objective_classification_comb if use_sample_weights else objective_classification_comb_nw

    study.optimize(lambda trial: optimize_fn(trial, data_dict),
                   n_trials=n_trials, callbacks=[print_progress])

    mean_rows = []
    std_rows = []
    for t in study.trials:
        row_mean = {
            "model": model_name,
            "type": "weighted" if use_sample_weights else "not weighted",
            "accuracy": t.user_attrs["overall_acc_mean"],
            "sensitivity": t.user_attrs["sensitivity_mean"],
            "specificity": t.user_attrs["specificity_mean"],
            "config": t.params
        }
        row_std = {
            "model": model_name,
            "type": "weighted" if use_sample_weights else "not weighted",
            "accuracy": t.user_attrs["overall_acc_std"],
            "sensitivity": t.user_attrs["sensitivity_std"],
            "specificity": t.user_attrs["specificity_std"],
            "config": t.params
        }
        mean_rows.append(row_mean)
        std_rows.append(row_std)

    columns = ["model", "type", "accuracy", "sensitivity", "specificity", "config"]
    mean_df = pd.DataFrame(mean_rows, columns=columns)
    std_df = pd.DataFrame(std_rows, columns=columns)

    best_trial = study.best_trial
    optimal_configuration = {
        "value": best_trial.value,
        "params": best_trial.params,
        "user_attrs": best_trial.user_attrs
    }

    result_dict = {
        "mean_metrics": mean_df,
        "std_metrics": std_df,
        "optimal_configuration": optimal_configuration
    }
    save_results_pickle(result_dict, model_name, use_sample_weights)
    return result_dict

comb

In [13]:
def objective_classification_fitbit_nw(trial, data_dict):
    """
    Objective function for fitbit_class using LSTM on non-weighted data.
    Searched hyperparameters: lr, dropout, batch_size.
    Fixed parameters: hidden_size=32, num_spochs=7.
    """
    set_seed(42)
    splits = get_stratified_cv_splits(
        data_dict['train'], subject_id="PatientID", target_var="is_SI", n_splits=5
    )
    
    overall_acc_list = []
    overall_sens_list = []
    overall_spec_list = []
    
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    dropout_val = trial.suggest_float("dropout", 0.1, 0.5)
    batch_size = trial.suggest_int("batch_size", 16, 64, step=16)
    
    # Fixed parameters
    hidden_size = 32
    num_spochs = 7  # fixed number of epochs
    
    for train_split, val_split in splits:
        # Force non-weighted
        train_df, _ = create_subject_dataset(train_split, outcome_col="is_SI", use_weights=False)
        val_df, _   = create_subject_dataset(val_split, outcome_col="is_SI", use_weights=False)
        
        X_train = np.stack(train_df["X"].values, axis=0)
        y_train = train_df["is_SI"].values.astype(np.float32)
        
        X_val = np.stack(val_df["X"].values, axis=0)
        y_val = val_df["is_SI"].values.astype(np.float32)
        
        X_train = np.transpose(X_train, (0, 2, 1))
        X_val   = np.transpose(X_val, (0, 2, 1))
        n_subjects, seq_len, input_size = X_train.shape

        class FitbitClassLSTM(nn.Module):
            def __init__(self, input_size, hidden_size, dropout):
                super(FitbitClassLSTM, self).__init__()
                self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
                self.dropout = nn.Dropout(dropout)
                self.fc = nn.Linear(hidden_size, 1)
            def forward(self, x):
                lstm_out, (h_n, _) = self.lstm(x)
                h_last = h_n[-1]
                out = self.dropout(h_last)
                out = self.fc(out)
                return out

        model = FitbitClassLSTM(input_size, hidden_size, dropout_val).to(device)
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
        
        optimizer = optim.Adam(model.parameters(), lr=lr)
        loss_fn = nn.BCEWithLogitsLoss()
        
        train_dataset = torch.utils.data.TensorDataset(
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
        )
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
        
        model.train()
        for epoch in range(num_spochs):
            for X_batch, y_batch in train_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = loss_fn(outputs, y_batch)
                loss.backward()
                optimizer.step()
        
        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            logits = model(X_val_tensor)
            probs = torch.sigmoid(logits).cpu().numpy().reshape(-1)
            preds = (probs >= 0.5).astype(np.float32)
        fold_acc = np.mean(preds == y_val)
        TP = np.sum((preds == 1) & (y_val == 1))
        FN = np.sum((preds == 0) & (y_val == 1))
        sensitivity = TP / (TP + FN) if (TP + FN) > 0 else np.nan
        TN = np.sum((preds == 0) & (y_val == 0))
        FP = np.sum((preds == 1) & (y_val == 0))
        specificity = TN / (TN + FP) if (TN + FP) > 0 else np.nan
        
        overall_acc_list.append(fold_acc)
        overall_sens_list.append(sensitivity)
        overall_spec_list.append(specificity)
    
    mean_acc = np.mean(overall_acc_list)
    std_acc = np.std(overall_acc_list)
    mean_sens = np.mean(overall_sens_list)
    std_sens = np.std(overall_sens_list)
    mean_spec = np.mean(overall_spec_list)
    std_spec = np.std(overall_spec_list)
    
    trial.set_user_attr("overall_acc_mean", mean_acc)
    trial.set_user_attr("overall_acc_std", std_acc)
    trial.set_user_attr("sensitivity_mean", mean_sens)
    trial.set_user_attr("sensitivity_std", std_sens)
    trial.set_user_attr("specificity_mean", mean_spec)
    trial.set_user_attr("specificity_std", std_spec)
    
    return 1 - mean_acc

def objective_classification_fitbit(trial, data_dict):
    """
    Objective function for fitbit_class using LSTM.
    Tunable hyperparameters:
      - lr
      - hidden_size
      - dropout
      - num_spochs
    """
    set_seed(42)
    splits = get_stratified_cv_splits(data_dict['train'], subject_id="PatientID", target_var="is_SI", n_splits=5)
    
    overall_acc_list = []
    overall_sens_list = []
    overall_spec_list = []
    
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    hidden_size = trial.suggest_int("hidden_size", 16, 128, step=16)
    dropout_val = trial.suggest_float("dropout", 0.1, 0.5)
    num_spochs = trial.suggest_int("num_spochs", 5, 10)
    batch_size = 32  # fixed
    
    for train_split, val_split in splits:
        train_df, _ = create_subject_dataset(train_split, outcome_col="is_SI")
        val_df, _ = create_subject_dataset(val_split, outcome_col="is_SI")
        
        X_train = np.stack(train_df["X"].values, axis=0)  # (n_subjects, n_features, seq_len)
        y_train = train_df["is_SI"].values.astype(np.float32)
        
        X_val = np.stack(val_df["X"].values, axis=0)
        y_val = val_df["is_SI"].values.astype(np.float32)
        
        # Transpose to (n_subjects, seq_len, n_features)
        X_train = np.transpose(X_train, (0, 2, 1))
        X_val = np.transpose(X_val, (0, 2, 1))
        n_subjects, seq_len, input_size = X_train.shape
        
        # Define LSTM model for fitbit classification
        class FitbitClassLSTM(nn.Module):
            def __init__(self, input_size, hidden_size, dropout):
                super(FitbitClassLSTM, self).__init__()
                self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
                self.dropout = nn.Dropout(dropout)
                self.fc = nn.Linear(hidden_size, 1)
            def forward(self, x):
                lstm_out, (h_n, _) = self.lstm(x)
                h_last = h_n[-1]
                out = self.dropout(h_last)
                out = self.fc(out)
                return out

        model = FitbitClassLSTM(input_size, hidden_size, dropout_val).to(device)
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
        
        optimizer = optim.Adam(model.parameters(), lr=lr)
        loss_fn = nn.BCEWithLogitsLoss()
        
        train_dataset = torch.utils.data.TensorDataset(
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
        )
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
        
        model.train()
        for epoch in range(num_spochs):  # using num_spochs instead of fixed epochs
            for X_batch, y_batch in train_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = loss_fn(outputs, y_batch)
                loss.backward()
                optimizer.step()
        
        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            logits = model(X_val_tensor)
            probs = torch.sigmoid(logits).cpu().numpy().reshape(-1)
            preds = (probs >= 0.5).astype(np.float32)
        fold_acc = np.mean(preds == y_val)
        TP = np.sum((preds == 1) & (y_val == 1))
        FN = np.sum((preds == 0) & (y_val == 1))
        sensitivity = TP / (TP + FN) if (TP + FN) > 0 else np.nan
        TN = np.sum((preds == 0) & (y_val == 0))
        FP = np.sum((preds == 1) & (y_val == 0))
        specificity = TN / (TN + FP) if (TN + FP) > 0 else np.nan
        
        overall_acc_list.append(fold_acc)
        overall_sens_list.append(sensitivity)
        overall_spec_list.append(specificity)
    
    mean_acc = np.mean(overall_acc_list)
    std_acc = np.std(overall_acc_list)
    mean_sens = np.mean(overall_sens_list)
    std_sens = np.std(overall_sens_list)
    mean_spec = np.mean(overall_spec_list)
    std_spec = np.std(overall_spec_list)
    
    trial.set_user_attr("overall_acc_mean", mean_acc)
    trial.set_user_attr("overall_acc_std", std_acc)
    trial.set_user_attr("sensitivity_mean", mean_sens)
    trial.set_user_attr("sensitivity_std", std_sens)
    trial.set_user_attr("specificity_mean", mean_spec)
    trial.set_user_attr("specificity_std", std_spec)
    
    return 1 - mean_acc

def run_deep_search_fitbit_class(data_dict, use_sample_weights, model_name, n_trials=75):
    """
    Run deep search for fitbit_class.
    Selects the proper objective function based on the use_sample_weights flag.
    """
    data_dict["use_sample_weights"] = use_sample_weights
    optuna.logging.set_verbosity(optuna.logging.WARNING)

    def print_progress(study, trial):
        print(f"{model_name} trial {len(study.trials)}/{n_trials}\r", end="", flush=True)

    study = optuna.create_study(direction="minimize")
    optimize_fn = objective_classification_fitbit if use_sample_weights else objective_classification_fitbit_nw

    study.optimize(lambda trial: optimize_fn(trial, data_dict),
                   n_trials=n_trials, callbacks=[print_progress])

    mean_rows = []
    std_rows = []
    for t in study.trials:
        row_mean = {
            "model": model_name,
            "type": "weighted" if use_sample_weights else "not weighted",
            "accuracy": t.user_attrs["overall_acc_mean"],
            "sensitivity": t.user_attrs["sensitivity_mean"],
            "specificity": t.user_attrs["specificity_mean"],
            "config": t.params
        }
        row_std = {
            "model": model_name,
            "type": "weighted" if use_sample_weights else "not weighted",
            "accuracy": t.user_attrs["overall_acc_std"],
            "sensitivity": t.user_attrs["sensitivity_std"],
            "specificity": t.user_attrs["specificity_std"],
            "config": t.params
        }
        mean_rows.append(row_mean)
        std_rows.append(row_std)

    columns = ["model", "type", "accuracy", "sensitivity", "specificity", "config"]
    mean_df = pd.DataFrame(mean_rows, columns=columns)
    std_df = pd.DataFrame(std_rows, columns=columns)

    best_trial = study.best_trial
    optimal_configuration = {
        "value": best_trial.value,
        "params": best_trial.params,
        "user_attrs": best_trial.user_attrs
    }

    result_dict = {
        "mean_metrics": mean_df,
        "std_metrics": std_df,
        "optimal_configuration": optimal_configuration
    }
    save_results_pickle(result_dict, model_name, use_sample_weights)
    return result_dict

### Run Deep Search for Classification with weights

In [16]:
results_comb_class = run_deep_search_comb_class(comb_class_dict, use_sample_weights=True, model_name="comb_class_deep", n_trials=75)
results_comb_class["mean_metrics"]


Results saved to search/comb_class_deep_deep_search_results.pkl


,model,type,accuracy,sensitivity,specificity,config
0,comb_class_deep,weighted,0.763398,0.009412,0.999267,"{'lr': 9.27062449324707e-05, 'dropout': 0.4552..."
1,comb_class_deep,weighted,0.718146,0.091655,0.914286,"{'lr': 9.611670464299026e-05, 'dropout': 0.134..."
2,comb_class_deep,weighted,0.742201,0.156908,0.925275,"{'lr': 0.007287458567971986, 'dropout': 0.3569..."
3,comb_class_deep,weighted,0.761720,0.004679,0.998535,"{'lr': 4.903496748640037e-05, 'dropout': 0.343..."
4,comb_class_deep,weighted,0.758371,0.014118,0.991209,"{'lr': 2.4021044387147108e-05, 'dropout': 0.29..."
...,...,...,...,...,...,...
70,comb_class_deep,weighted,0.728784,0.182544,0.899634,"{'lr': 0.0007938829759058924, 'dropout': 0.260..."
71,comb_class_deep,weighted,0.762282,0.044487,0.986813,"{'lr': 0.00016201017991013098, 'dropout': 0.29..."
72,comb_class_deep,weighted,0.764512,0.039781,0.991209,"{'lr': 0.00019936384138497678, 'dropout': 0.31..."
73,comb_class_deep,weighted,0.763955,0.051518,0.986813,"{'lr': 0.00022303473310716408, 'dropout': 0.17..."


In [17]:
results_fitbit_class = run_deep_search_fitbit_class(fitbit_class_dict, use_sample_weights=True, model_name="fitbit_class_deep", n_trials=75)
results_fitbit_class["mean_metrics"]


Results saved to search/fitbit_class_deep_deep_search_results.pkl


,model,type,accuracy,sensitivity,specificity,config
0,fitbit_class_deep,weighted,0.490226,0.576635,0.462271,"{'lr': 1.1820409925225312e-05, 'hidden_size': ..."
1,fitbit_class_deep,weighted,0.761163,0.004651,0.997802,"{'lr': 0.0001547622268491406, 'hidden_size': 1..."
2,fitbit_class_deep,weighted,0.737748,0.088755,0.940659,"{'lr': 3.210643749578772e-05, 'hidden_size': 4..."
3,fitbit_class_deep,weighted,0.538554,0.383776,0.586813,"{'lr': 3.7523058456721776e-05, 'hidden_size': ..."
4,fitbit_class_deep,weighted,0.762277,0.002326,1.000000,"{'lr': 0.00011395742363510966, 'hidden_size': ..."
...,...,...,...,...,...,...
70,fitbit_class_deep,weighted,0.763955,0.018769,0.997070,"{'lr': 0.0001663799488249265, 'hidden_size': 1..."
71,fitbit_class_deep,weighted,0.763955,0.018769,0.997070,"{'lr': 0.00017296898246593756, 'hidden_size': ..."
72,fitbit_class_deep,weighted,0.763396,0.018769,0.996337,"{'lr': 0.00017847120939321913, 'hidden_size': ..."
73,fitbit_class_deep,weighted,0.760604,0.000000,0.998535,"{'lr': 0.00018117713776052256, 'hidden_size': ..."


### Run Deep Search for Classification without weights

In [18]:
results_comb_class = run_deep_search_comb_class(comb_class_dict, use_sample_weights=False, model_name="comb_class_deep", n_trials=75)
results_comb_class["mean_metrics"]


Results saved to search/comb_class_deep_nw_deep_search_results.pkl


,model,type,accuracy,sensitivity,specificity,config
0,comb_class_deep,not weighted,0.764513,0.028153,0.994872,"{'lr': 0.00016358797976479862, 'dropout': 0.14..."
1,comb_class_deep,not weighted,0.763396,0.009412,0.999267,"{'lr': 5.8754396545751666e-05, 'dropout': 0.11..."
2,comb_class_deep,not weighted,0.761720,0.000000,1.000000,"{'lr': 4.1535039949794044e-05, 'dropout': 0.37..."
3,comb_class_deep,not weighted,0.726548,0.161587,0.903297,"{'lr': 0.0025937976958127404, 'dropout': 0.372..."
4,comb_class_deep,not weighted,0.761720,0.000000,1.000000,"{'lr': 1.4795861732331276e-05, 'dropout': 0.23..."
...,...,...,...,...,...,...
70,comb_class_deep,not weighted,0.717634,0.189631,0.882784,"{'lr': 0.0007061443580878221, 'dropout': 0.396..."
71,comb_class_deep,not weighted,0.764510,0.011710,1.000000,"{'lr': 7.830546659454098e-05, 'dropout': 0.359..."
72,comb_class_deep,not weighted,0.765074,0.046785,0.989744,"{'lr': 0.00013777799219093285, 'dropout': 0.34..."
73,comb_class_deep,not weighted,0.765074,0.046785,0.989744,"{'lr': 0.00013646870644887075, 'dropout': 0.34..."


In [19]:
results_fitbit_class = run_deep_search_fitbit_class(fitbit_class_dict, use_sample_weights=False, model_name="fitbit_class_deep", n_trials=75)
results_fitbit_class["mean_metrics"]



Results saved to search/fitbit_class_deep_nw_deep_search_results.pkl


,model,type,accuracy,sensitivity,specificity,config
0,fitbit_class_deep,not weighted,0.756706,0.002326,0.992674,"{'lr': 7.922039201138631e-05, 'dropout': 0.405..."
1,fitbit_class_deep,not weighted,0.761163,0.002326,0.998535,"{'lr': 0.00030929653350372435, 'dropout': 0.25..."
2,fitbit_class_deep,not weighted,0.756705,0.072640,0.970696,"{'lr': 0.005983293892025289, 'dropout': 0.2982..."
3,fitbit_class_deep,not weighted,0.638585,0.221122,0.768498,"{'lr': 3.978021711839062e-05, 'dropout': 0.350..."
4,fitbit_class_deep,not weighted,0.756697,0.023338,0.986081,"{'lr': 0.008579417466902212, 'dropout': 0.4561..."
...,...,...,...,...,...,...
70,fitbit_class_deep,not weighted,0.750012,0.063311,0.964835,"{'lr': 0.0028354454973709667, 'dropout': 0.171..."
71,fitbit_class_deep,not weighted,0.762837,0.025663,0.993407,"{'lr': 0.0009601436128208786, 'dropout': 0.184..."
72,fitbit_class_deep,not weighted,0.763395,0.025663,0.994139,"{'lr': 0.0009763774609188299, 'dropout': 0.208..."
73,fitbit_class_deep,not weighted,0.760608,0.014008,0.994139,"{'lr': 0.0006862805508258147, 'dropout': 0.217..."
